In [0]:
# -*- conding: utf-8 -*-
import numpy   as np
import random
import copy

In [0]:
class Environment:
    #게임판 구성, 안되는 상태, 완료된 상태, 보상 정의,
    #게임판 핸드만 두장 줌, 베팅 한 번씩 물어봄. 리레이즈x

    # 덱을 만듬. 턴, 판돈, 플레이어가 베팅한 횟수 설정


    def __init__(self):
        self.done = False #True면 게임 종료
        #self.winner = 0
        self.reward = 0
        self.turn = 0
        self.table = 0
        self.player_calls =0
        self.lowlimit = 200
        self.maxlimit = 2000
        self.bet1 = -2
        self.bet2 = -2

        deck = [(suit, i) for suit in ["s", "h", "d", "c"] for i in range(2, 15)]  ##카드 덱 만들기, 튜플로 구성된 리스트
        random.shuffle(deck)
        self.deck = deck

    # 두 플레이어에게 분배할 네장을 뽑음
        #self.hands = [self.deck[0], self.deck[1], self.deck[2], self.deck[3]]

    # 보상을 정의  handcheck, ai_action은 agent가 준다.
    def pf_reward(self, handcheck, ai_action):
        #print('hc, a', handcheck, ai_action)
        if (handcheck == 1) and (ai_action > 0):
            #print('ai folded')
            # if ai_action >= 0:  # 콜, 레이즈, 체크
            rwd = -1
        elif (handcheck == 1) and (ai_action < 0):  # 폴드한 경우
            rwd = +1

        elif (handcheck != 1) and (ai_action > 0):
            # if ai_action >= 0:  # 콜, 레이즈, 체크
            rwd = +1
        else:  # 폴드한 경우
            rwd = -1

        return rwd

    def move(self, p1,p2): #, player):
        #각플레이어가 선택한 행동 table에 반영하고 게임 상태 판단
        #p1 = 1, p2 = -1
        #각 플레이어는 행동을 선택하는 메서드를 가짐 selct_action
        #if player == 1:
        self.bet1 = p1.select_action(env)#, player)
        if self.bet1 == -1: #폴드한 경우
          pass
        else:
          self.table += self.bet1
        self.player_calls += 1
        #else:
        self.bet2 = p2.select_action(env)#, player)
        if self.bet2 > self.lowlimit: #리레이즈시
          self.table += self.bet2
          self.bet1 = p1.select_action(env)#, player) # 한번 더 행동을 받음
          if self.bet1 == -1:
            pass
          else:
            self.table += self.bet1
          self.player_calls += 1
        else:
          if self.bet2 == -1:
            pass
          else:
            self.table += self.bet2
          self.player_calls += 1

         
        #self.table += bet1 +bet2
        #print('table: ',self.table)
        #게임 상태가 종료인지 판단, 누가 이겼는지 체크
      
        self.end_check(p1,p2)
        self.player_calls =0
        return self.reward, self.done

    #현재 베팅 가능한 상태 수집.
    def get_action(self):
        observation = [-1,0]
        #table이 0이 아니다. -> 누군가 베팅을 했다면 lowlimit 이 그 두배로 바뀜
        #print('테이블 상황:', self.table)
        if self.table > 0:
          if self.table*2 > 2000:
            self.lowlimit = 2000
          else:
            self.lowlimit =  self.table*2
        #print('로우리밋 상황', self.lowlimit)
        for i in range(self.lowlimit, self.maxlimit+1, 100): #이 부분 변경하면 경우의 수를 줄일 수 있음.
            observation.append(i)
        return observation

    #게임 종료 판단
    def end_check(self, p1,p2):
        if self.player_calls == 2:
            # 1. agent 가 player보다 카드가 좋다.
            #판돈에 플레이어의 베팅을 반영 및 표시
            if (self.bet1 == -1): #누군가 폴드하면 상대가 
              p2.bal += self.table
              #p1.bal -= 1
              self.table = 0
              self.reward = -1 # 승자는 p2
              self.done = True
              #self.player_calls =0
            elif (self.bet2 == -1): # 누군가 폴드한 경우
              p1.bal += self.table
              #p2.bal -= 1
              self.table = 0
              self.reward = 1 #승자는 p1
              self.done = True
              
              #self.player_calls =0
            else:
              #print('winner checking', p1.hand1, p1.hand2, p2.hand1, p2.hand2)
              #print(self.hand_check(p1.hand1, p1.hand2))
              #print(self.hand_check(p2.hand1, p2.hand2))
              #print(self.bet1, self.bet2)
              if self.hand_check(p1.hand1, p1.hand2) > self.hand_check(p2.hand1, p2.hand2):
                  self.reward = 1
                  p1.bal += self.table  # 테이블의 돈을 가져감
                  self.table = 0
                  self.done = True
              # 2. agent 가 player보다 카드가 안 좋다.
              elif self.hand_check(p1.hand1, p1.hand2) < self.hand_check(p2.hand1, p2.hand2):
                  self.reward = -1
                  p2.bal += self.table  # 테이블의 돈을 가져감
                  self.table = 0
                  self.done = True
              # 3. agent 가 player보다 카드가 같다. -> 무승부 판돈 분할
              else:
                  self.reward = 0
                  p1.bal += self.table/2
                  p2.bal += self.table/2
                  self.table = 0
                  self.done = True # 돈은 그대로 남아있음.
        else:
            pass
        return

    def print_poker(self, player):
        print('\n------------------Turn : %d-------------------\n'%self.turn)
        print('Table Money: %d'%self.table)
        print('Now Bet: %s'%player.name)

    # 핸드 순위 확인
    def hand_check(self, hand1, hand2):  # 핸드체크, 프레플랍 단계 먼저 자신에게 주어진 카드 정보를 분석한다.
        premium = [('p', 14, 14), ('p', 13, 13), ('p', 12, 12), ('s', 14, 13), ('p', 11, 11), ('s', 14, 12),
                   ('s', 13, 12), ('s', 14, 11), ('s', 13, 11), ('p', 10, 10), ('o', 14, 13), ('s', 14, 10),
                   ('s', 12, 11), ('s', 13, 10), ('s', 12, 10), ('s', 11, 10), ('p', 9, 9)]
        special = [('o', 14, 12), ('s', 14, 9), ('o', 13, 12), ('p', 8, 8), ('s', 13, 9), ('s', 10, 9),
                   ('s', 14, 8),
                   ('s', 12, 9), ('s', 11, 9), ('o', 14, 11), ('s', 14, 5), ('p', 7, 7), ('s', 14, 7),
                   ('o', 13, 11),
                   ('s', 14, 4), ('s', 14, 3), ('s', 14, 6)]
        good = [('o', 12, 11), ('p', 6, 6), ('s', 13, 8), ('s', 10, 8), ('s', 14, 2), ('s', 9, 8), ('s', 11, 8),
                ('o', 14, 10), ('s', 12, 8), ('s', 13, 7), ('o', 13, 10), ('p', 5, 5), ('o', 11, 10), ('s', 8, 7),
                ('o', 12, 10), ('p', 4, 4), ('p', 3, 3), ('p', 2, 2)]
        # 숫자 비교 후 pair인지 확인(pair이면 p, 아니면 문자 확인으로 넘어감), 문자 확인해서 일치하는지 확인(불일치 하면 o, 일치하면 s ),  숫자 조합해서 하나의 튜플로 제작
        if hand1[1] == hand2[1]:
            tup1 = 'p'
        else:
            if hand1[0] == hand2[0]:
                tup1 = 's'
            else:
                tup1 = 'o'

        if hand1[1] > hand2[1]:  # 숫자 큰 게 앞으로 오게 함
            tup2, tup3 = hand1[1], hand2[1]
        else:
            tup2, tup3 = hand2[1], hand1[1]

        tup = (tup1, tup2, tup3)  # 튜플 생성
        #print('tup', tup)
        if premium.count(tup) == 1:
            return 4  # ('premium')
        elif special.count(tup) == 1:
            return 3  # ('special')
        elif good.count(tup) == 1:  # 판단
            return 2  # ('good')
        else:
            return 1  # ('fold')

In [0]:
class Human_player:
    
    
    def __init__(self):
        self.name = 'Human Player'
        self.bal = 50000
        self.hand1 = 0
        self.hand2 = 0
    
    
    def select_action(self, env):#,player):
        while True:
            available_action = env.get_action()
            if (env.bet1 > 0) and (env.bet2> 2*env.bet1): #리레이즈의 상황이라면 
              available_action = [-1, env.bet2-env.bet1] #폴드하거나 콜만 함
            print("possible actions = {}".format(available_action))
            print("{} Hands: {}{}".format(self.name, self.hand1, self.hand2))
            action = int(input(f"Select action(human)(fold == -1, call/check = 0 , {env.lowlimit} <= raise<= {env.maxlimit}  : "))
            if action in available_action:
              if action == 0: # 콜/체크라면 상대가 제시한 만큼 내게 됨. 추후 라운드 추가시 변경
                self.bal -= env.table
                env.table += env.table 
              self.bal -= action
              return action
            else:
                print("Wrong Action. Try Again")
        return


In [0]:
class Random_player:
    
    def __init__(self):
        self.name = 'Random Player'
        self.bal = 50000
        self.hand1 = 0
        self.hand2 = 0

    def select_action(self, env):

        available_action = env.get_action()
        if (env.bet1 > 0) and (env.bet2> 2*env.bet1): #리레이즈의 상황이라면 
          available_action = [-1, env.bet2-env.bet1] #폴드하거나 콜만 함
        #print("possible actions = {}".format(available_action))
        #print("{} Hands: {}{}".format(self.name, self.hand1, self.hand2))
        action = random.choice(available_action)
        #print(action)
        if action == 0: # 콜/체크라면 상대가 제시한 만큼 내게 됨. 추후 라운드 추가시 변경
          self.bal -= env.table
          env.table += env.table
        if action == -1:
          pass
        else:
          self.bal -= action
        return action

In [0]:
class Monte_Carlo_player:
  def __init__(self):
    self.name = "MC player"
    self.num_playout = 1
    self.bal = 50000
    self.hand1 = 0
    self.hand2 = 0
    

  def select_action(self, env):#, player)
    #가능한 행동 조사
    available_action = env.get_action()
    if (env.bet1 > 0) and (env.bet2> 2*env.bet1): #리레이즈의 상황이라면 
      available_action = [-1, env.bet2-env.bet1] #폴드하거나 콜만 함
    V = np.zeros(len(available_action))

    temp_p1 = Random_player
    temp_p1.hand1 = p1.hand1
    temp_p1.hand2 = p1.hand2
    temp_p1.name = p1.name
    temp_p1.bal = p1.bal
    temp_p2 = Random_player()
    temp_p2.hand1 = p2.hand1
    temp_p2.hand2 = p2.hand2
    temp_p2.name = p2.name
    temp_p2.bal = p2.bal

    for i in range(len(available_action)):

      #플레이아웃을 1000번 반복
      for j in range(self.num_playout):

        #지금 상태를 복사해서 블레이 아웃에 사용
        temp_env = copy.deepcopy(env)
        #플레이 아웃의 결과는 승리 플레이어의 값으로 반환
        #p1이 이기면 temp_env = 1, p2가 이기면 temp_env = -1
     
        self.playout(temp_env, available_action[i], temp_p1, temp_p2)#, player)
        if temp_p1.name == 'MC player':
          if temp_env.reward == 1:
            V[i] += temp_env.pf_reward(temp_env.hand_check(temp_p1.hand1, temp_p1.hand2), available_action[i])#i#temp_env.lowlimit#available_action[i]#i#1#temp_env.bet2
        if temp_p2.name == 'MC player':
          if temp_env.reward == -1:
            V[i] += temp_env.pf_reward(temp_env.hand_check(temp_p1.hand1, temp_p1.hand2), available_action[i])#i#temp_env.lowlimit#available_action[i]#i#1#temp_env.bet2
    
    action = available_action[np.argmax(V)]
    if action == 0: # 콜/체크라면 상대가 제시한 만큼 내게 됨. 추후 라운드 추가시 변경
          self.bal -= env.table
          env.table += env.table
    if action == -1:
      pass
    else:
      self.bal -= action
    return action

  #플레이아웃 재귀 함수
  # 게임이 종료 상태(승/패/비김)가 될 때까지 행동을 임의로 선택하는 것을 반복
  #플레이어는 계속 바뀌기 때문에 (-)를 곱해서 -1, 1 ,-1 순서로 변하게 함
  def playout(self, temp_env, action,temp_p1, temp_p2):#, player):
    #판돈에 플레이어의 베팅을 반영 및 표시
    #print('playout start: action =', action)
    if temp_p1.name == 'MC player':
      temp_env.bet1 = action
      temp_env.table += temp_env.bet1
    if temp_p2.name == 'MC player':
      temp_env.bet2 = action
      temp_env.table += temp_env.bet2
    temp_env.player_calls += 1
    
    #print(temp_env.table, temp_env.reward)
     #게임 종료 체크 
    if temp_env.player_calls == 2:
      #temp_env.done = True
      #print('playout end_check')
      temp_env.end_check(temp_p1, temp_p2)
      temp_env.player_calls =0
      #print(available_action)
      return #player

    else:
      #가능한 행동 조사 
      #a무작위로 행동을 선택
      if temp_p1.name == 'MC player':
        temp_env.bet2 = temp_p2.select_action(temp_env)
        temp_env.table += temp_env.bet2
      if temp_p2.name == 'MC player':
        temp_env.bet1 = temp_p1.select_action(temp_env)
        temp_env.table += temp_env.bet2
      temp_env.player_calls += 1

In [0]:
#p1 = Monte_Carlo_player()
p1 = Random_player()
#p1 = Human_player()
#p2 = Random_player()
p2 = Monte_Carlo_player()
#p2 = Human_player()
# 지정된 게임 수를 자동으로 두게 할 것인지 한 게임씩 두게 할 것인지 결정
# auto = True : 지정된 판 수 (games)를 자동으로 진행
# auto = Flase: 한 게임씩 진행
auto = True

#auto 모드의 게임 수
games = 10000

#각 플레이어의 승리 횟수를 저장
p1_score = 0
p2_score = 0
draw_score = 0

In [30]:
if auto:

    #자동모드 실행
    for j in range(games):
        np.random.seed(j)
        env = Environment()
        p1.hand1, p1.hand2 = env.deck.pop(), env.deck.pop()
        p2.hand1, p2.hand2 = env.deck.pop(), env.deck.pop()
        print('\n------------------------------------------------------------------------------------------------------------------')
        print("p1 player, money, hand : {} {} {}{}".format(p1.name, p1.bal, p1.hand1, p1.hand2))
        print("p2 player, money hand: {} {} {}{}".format(p2.name, p2.bal, p2.hand1, p2.hand2))
        
        if (p1.bal < 0) or (p2.bal <0):
          print('Player Balance Under 0, End this Game.\n')
          break
        for i in range(10000):
            # p1, p2 번갈아가면서 게임을 진행
            # p1(1) -> p2(-1)
            sum = p1.bal+p2.bal
           
            reward, done = env.move(p1,p2) #,(-1)**i)
            print('after move',sum, env.bet1, env.bet2)
            if sum != 100000:
              print('after move',sum, env.bet1, env.bet2)
              input()
            print('reward', reward)
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    print("j = {} winner is p1({})".format(j, p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("j = {} winner is p2({})".format(j, p2.name))
                    p2_score += 1
                else:
                    print("j = {} winner is draw".format(j))
                    draw_score += 1
                break
    print("\ntable= {} p1({}) = {} p2({}) = {} draw = {}".format(env.table, p1.name, p1_score, p2.name, p2_score, draw_score))
    print("p1.bal= {} p2.bal={} sum = {}".format(p1.bal, p2.bal, p1.bal+p2.bal))
    
else:
    #한 게임씩 진행하는 수동 모드
    while True:
        #env = Environment()
        env = Environment()
        p1.hand1, p1.hand2 = env.deck.pop(), env.deck.pop()
        p2.hand1, p2.hand2 = env.deck.pop(), env.deck.pop()
        print('------------------------------------------------------------------------------------------------------------------')
        print("p1 player, money, hand : {} {} {}{}".format(p1.name, p1.bal, p1.hand1, p1.hand2))
        print("p2 player, money hand: {} {} {}{}".format(p1.name, p2.bal, p2.hand1, p2.hand2))
        #print('------------------------------------------------------------------------------------------------------------------')
        #env.print = True

        for i in range(10000):
            # p1, p2 번갈아가면서 게임을 진행
            # p1(1) -> p2(-1)
            reward, done = env.move(p1, p2, (-1)**i)
            print('reward', reward)
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    print("Winner is p1({})".format(p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("Winner is p2({})".format(p2.name))
                    p2_score += 1
                else:
                    print("No Winner")
                    draw_score += 1
                break

        print("final result")
        env.print_poker(p1)

        print("final result")
        env.print_poker(p2)

        # one more?
        answer = input("more Game? (y/n)")
        print("table= {} p1({}) = {} p2({}) = {} draw = {}".format(env.table, p1.name, p1_score, p2.name, p2_score, draw_score))
        if answer == 'y':
            pass
        else:
            break

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
------------------------------------------------------------------------------------------------------------------
p1 player, money, hand : Random Player 9700 ('h', 10)('c', 7)
p2 player, money hand: MC player 90300 ('s', 5)('c', 3)
after move 100000 900 -1
reward 1
j = 1927 winner is p1(Random Player)

------------------------------------------------------------------------------------------------------------------
p1 player, money, hand : Random Player 9700 ('c', 14)('h', 13)
p2 player, money hand: MC player 90300 ('c', 9)('d', 10)
after move 100000 1300 -1
reward 1
j = 1928 winner is p1(Random Player)

------------------------------------------------------------------------------------------------------------------
p1 player, money, hand : Random Player 9700 ('s', 10)('h', 2)
p2 player, money hand: MC player 90300 ('c', 5)('d', 8)
after move 100000 1100 -1
reward 1
j = 1929 winner is p1(Random Player)

---------------------------------------------